## Mount the Drive, and Change to Google Drive Folder

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

# %cd /content/drive/MyDrive/MSc.-Dissertations/1/Files
# %ls

## Import Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, utils, losses, optimizers
from keras.wrappers import scikit_learn
from keras.models import *
from keras.layers import *
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils, image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

import random
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

## Count the Number of Files, and Take Random Samples from the Image Files

In [2]:
# !ls street_view
# count how many files and write the filenames into a file
# !ls street_view -1 | wc -l 
# !ls street_view/*.jpg > flist.txt
flist_old = list(pd.read_csv('flist.txt', header = None)[0])
flist = []
change_names = list(map(lambda x: flist.append(f'street_view/{x}'), flist_old))

# Set seed so sample is reproducible 
# random.seed(99)  # set this to an integer value!!!
# nsamp = 100
# flist_sub = random.sample(flist, nsamp)
# flist = flist_sub

# print(flist)

## Overview of the `properties` Dataset

Read the `properties` dataset first, and make sure that `property type` is a categorical variable.

In [3]:
properties = pd.read_csv('properties.csv')
properties_juny12 = pd.read_csv('properties_juny12.csv')
properties_full = pd.concat([properties, properties_juny12])
properties = properties_full
properties.propertyType = properties.propertyType.astype('category')
properties.head()

,Unnamed: 0,address,propertyType,bedrooms,detailUrl,location_lat,location_lng,property_id
0,0,"12, Gorsey Brigg, Dronfield Woodhouse, Dronfie...",Terraced,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29986,-1.49446,60d9dd15-c5a0-4d9c-a341-a1d47add49d5
1,0,"5, Highgate Lane, Dronfield, Derbyshire S18 1UB",Detached,4.0,https://www.rightmove.co.uk/house-prices/detai...,53.29135,-1.45975,4a586e80-181a-4b82-b5c3-2d789436bb14
2,0,"125, Gosforth Lane, Dronfield, Derbyshire S18 1RB",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29763,-1.47573,93680b6c-237e-44d3-8f40-959a14b80cad
3,0,"80, Shakespeare Crescent, Dronfield, Derbyshir...",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29259,-1.45644,5d49758b-f148-4d06-bbae-3eb23f5c68fb
4,0,"21, Gainsborough Road, Dronfield, Derbyshire S...",Detached,NaN,https://www.rightmove.co.uk/house-prices/detai...,53.29740,-1.48503,4645f5eb-de7c-474f-8d7e-b59fa8c55f19


## A Subset of the `properties` Dataset

As random samples of images have been obtained previously, a subset of the whole `properties` dataset could hence be formulated by selecting the rows of the whole `properties` dataset corresponding to the selected samples.

In [19]:
flist_id = list(map(lambda string: string[16 : -4], flist))
properties_sub = pd.DataFrame(properties.loc[properties['property_id'].isin(flist_id)])
properties_sub = properties_sub.drop_duplicates(['location_lat', 'location_lng'])
flist_id = list(properties_sub.property_id)
flist_new = []
change_names_new = list(map(lambda x: flist_new.append(f'street_view/gsv_{x}.jpg'), flist_id))

In [32]:
import os

# List of image directories
image_dirs = list(map(lambda string: string[0 : 12], flist_new))

# List of files to keep
files_to_keep = list(map(lambda string: string[12 : ], flist_new))

# Remove files not in the list
for image_dir in image_dirs:
    for filename in os.listdir(image_dir):
        if filename not in files_to_keep:
            os.remove(os.path.join(image_dir, filename))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Robin\AppData\Local\Temp\ipykernel_11280\52720562.py", line -1, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\ProgramData\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\ProgramData\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1012, in structured_traceback
    return VerboseTB.structured_traceback(
  File "c:\ProgramData\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 865, in structured_tracebac

In [ ]:
import os
import shutil

# List of image directories and corresponding labels
image_dirs = ['dir1', 'dir2', 'dir3']
labels = ['label1', 'label2', 'label3']

# Create folders for each label
for label in labels:
    os.makedirs(label, exist_ok = True)

# Move images to corresponding folders
for i, image_dir in enumerate(image_dirs):
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            shutil.move(os.path.join(image_dir, filename), os.path.join(labels[i], filename))

The original data should be splitted into training and testing sets, and the testing set contains 30% of the original data.

In [ ]:
flist_train, flist_test, propertyType_train, propertyType_test = train_test_split(
    flist_new, # image directories
    properties_sub.propertyType, # property types
    test_size = 0.3)

In [ ]:
Img_list_train = list(map(lambda x: np.asarray(Image.open(x).resize((128, 128), Image.LANCZOS).filter(ImageFilter.FIND_EDGES)), flist_train))
Img_list_test = list(map(lambda x: np.asarray(Image.open(x).resize((128, 128), Image.LANCZOS).filter(ImageFilter.FIND_EDGES)), flist_test))
Img_array_train = np.asarray(Img_list_train)
Img_array_test = np.asarray(Img_list_test)

For categorical variables, one-hot encoder is introduced.

In [ ]:
dummy_propertyType_train = pd.get_dummies(propertyType_train)
dummy_propertyType_test = pd.get_dummies(propertyType_test)
propertyType_test_fac = np.argmax(np.array(dummy_propertyType_test), axis = 1) 

## Multi-Class Classification Using Neural Network

### Multi-Layer Perceptron (MLP) model

In [ ]:
def mlp(output_dim):

    '''
    Creates a multi-layer perceptron neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Flatten())
    model.add(Dense(128, activation = tf.nn.leaky_relu))
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    loss = keras.losses.CategoricalCrossentropy()
    weights = np.array(len(propertyType_train) / propertyType_train.value_counts())
    loss.weighted = weights
    model.compile(loss = loss, optimizer = keras.optimizers.Adam(), metrics = ['accuracy'])
    return model

In [ ]:
MLP = mlp(dummy_propertyType_train.shape[1])
MLP.fit(ImageDataGenerator().flow(Img_array_train, dummy_propertyType_train, batch_size = 256),
        epochs = 32, batch_size = 64, 
        validation_data = ImageDataGenerator().flow(Img_array_test, dummy_propertyType_test, batch_size = 256))

In [ ]:
propertyType_test_pred_MLP = np.argmax(MLP.predict(Img_array_test), axis = 1) 
MLP.evaluate(ImageDataGenerator().flow(Img_array_test, dummy_propertyType_test, batch_size = 256))

In [ ]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred_MLP)

## Convolutional Neural Network

In [ ]:
def cnn(output_dim):

    '''
    Creates a convolutional neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Conv2D(8, (7, 7), padding = 'same', activation = tf.nn.leaky_relu))
    model.add(Conv2D(4, (3, 3), padding = 'same', activation = tf.nn.leaky_relu))
    model.add(MaxPooling2D()) 
    model.add(Flatten())
    model.add(Dense(16, activation = tf.nn.leaky_relu))
    # model.add(Dropout(.5))
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    loss = keras.losses.CategoricalCrossentropy()
    weights = np.array(len(propertyType_train) / propertyType_train.value_counts())
    loss.weighted = weights
    model.compile(loss = loss, optimizer = keras.optimizers.Adam(), metrics = ['accuracy'])
    return model

In [ ]:
CNN = cnn(dummy_propertyType_train.shape[1])
CNN.fit(ImageDataGenerator().flow(Img_array_train, dummy_propertyType_train, batch_size = 32), 
        epochs = 32, batch_size = 64, 
        validation_data = ImageDataGenerator().flow(Img_array_test, dummy_propertyType_test, batch_size = 32))

In [ ]:
propertyType_test_pred_CNN = np.argmax(CNN.predict(Img_array_test), axis = 1) 
CNN.evaluate(ImageDataGenerator().flow(Img_array_test, dummy_propertyType_test, batch_size = 256))

In [ ]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred_CNN)

In [ ]:
# import dill
# dill.dump_session('Presetting.pkl')
# # dill.load_session('Presetting.pkl')